In [38]:
import pandas as pd
import numpy as np
import os, pdb, sys


# STEP 1: Ingest state/county employees and establishments data by naics code
path = 'C://Users//cshul//Downloads//cbp22co//cbp22co.txt'
df = pd.read_csv(path, sep = ",")

keeps = ['fipstate', 'fipscty', 'naics', 'emp', 'est']
df = df[keeps]
df.columns = ['state_fips', 'cty_fips', 'naics', 'employees', 'establishments']
df.head()

,state_fips,cty_fips,naics,employees,establishments
0,1,1,------,12409,948
1,1,1,11----,52,10
2,1,1,113///,39,7
3,1,1,1133//,39,7
4,1,1,11331/,39,7


In [39]:
# STEP 2: Ingest state and county names
path = 'C://Users//cshul//Downloads//US_FIPS_Codes.xlsx'
fips = pd.read_excel(path).iloc[1:].reset_index(drop = True)
fips.columns = ['state', 'county', 'state_fips', 'cty_fips']
fips['key'] = [i+j for i,j in zip(fips['state_fips'], fips['cty_fips'])]
fips.head()

,state,county,state_fips,cty_fips,key
0,Alabama,Autauga,01,001,01001
1,Alabama,Baldwin,01,003,01003
2,Alabama,Barbour,01,005,01005
3,Alabama,Bibb,01,007,01007
4,Alabama,Blount,01,009,01009


In [40]:
# STEP 3: Join state/county names based on key
df['state_fips'] = ['0' + str(i) if len(str(i)) < 2 else str(i) for i in df['state_fips']]

newCty = [] 
for i in df.index: 
    val = str(df['cty_fips'][i])
    while len(val) < 3: 
        val = '0' + val 
    newCty.append(val)
df['cty_fips'] = newCty
df['key'] = [i+j for i,j in zip(df['state_fips'], df['cty_fips'])]

df.head()

,state_fips,cty_fips,naics,employees,establishments,key
0,01,001,------,12409,948,01001
1,01,001,11----,52,10,01001
2,01,001,113///,39,7,01001
3,01,001,1133//,39,7,01001
4,01,001,11331/,39,7,01001


In [41]:
# STEP 4: JOIN 
df = pd.merge(df, fips[['state', 'county', 'key']], how = 'left', on = 'key')
df = df[['key', 'state_fips', 'cty_fips', 'state', 'county', 'naics', 'employees', 'establishments']]
df.head()

,key,state_fips,cty_fips,state,county,naics,employees,establishments
0,01001,01,001,Alabama,Autauga,------,12409,948
1,01001,01,001,Alabama,Autauga,11----,52,10
2,01001,01,001,Alabama,Autauga,113///,39,7
3,01001,01,001,Alabama,Autauga,1133//,39,7
4,01001,01,001,Alabama,Autauga,11331/,39,7


In [42]:
df = df[~df['state'].isna()].reset_index(drop = True)
df['yr'] = 2022

In [43]:
df.to_csv(os.getcwd() + '\\prepared_data_2022.csv', index = None)